In [1]:

import seaborn as sb
import matplotlib as mp
import os
import glob
import pandas as pd
import tqdm
from DataLoader_pytorch import window_torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch

/Users/hyunsucksong/opt/anaconda3/envs/songforgpu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_size, shift = 10, 5
train = []
test = []
val = []

using_columns = ['Time[s]', 'TE1', 'E.RPM',
       'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar','E.Id','RPM', 'P_UD']


#x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar']
x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Pwr' ]
print(len(x_columns))


y_columns = ['P_UD']



5


In [3]:
train_datapath = 'csv_final/train'
test_datapath = "csv_final/test"
val_datapath = "csv_final/val"
train_data = os.listdir(train_datapath)
test_data = os.listdir(test_datapath)
val_data = os.listdir(val_datapath)


In [4]:
num = 0
for csv_file in tqdm.tqdm(train_data) : 
    
    df_cut = pd.read_csv('csv_final/train/' + csv_file)[using_columns]
    train.append(df_cut)
    if num > 50 :
        break
    num = num + 1

'''for csv_file in tqdm.tqdm(test_data) : 
    
    df_cut = pd.read_csv('csv_final/test/' + csv_file)[using_columns]
    test.append(df_cut)
    
for csv_file in tqdm.tqdm(val_data) : 
    
    df_cut = pd.read_csv('csv_final/val/' + csv_file)[using_columns]
    val.append(df_cut)'''

 19%|████████                                  | 51/267 [00:02<00:11, 18.17it/s]


"for csv_file in tqdm.tqdm(test_data) : \n    \n    df_cut = pd.read_csv('csv_final/test/' + csv_file)[using_columns]\n    test.append(df_cut)\n    \nfor csv_file in tqdm.tqdm(val_data) : \n    \n    df_cut = pd.read_csv('csv_final/val/' + csv_file)[using_columns]\n    val.append(df_cut)"

In [5]:
print(len(train))
print(train)

52
[      Time[s]     TE1   E.RPM  E.Vlt  E.Iq  E.Tmp  E.Pwr   E.Tar  E.Id  RPM  \
0       0.000  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
1       0.001  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
2       0.002  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
3       0.003  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
4       0.004  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
...       ...     ...     ...    ...   ...    ...    ...     ...   ...  ...   
9213    9.213  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9214    9.214  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9215    9.215  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9216    9.216  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9217    9.217  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   

       P_UD  
0     1.543  
1     1.545  
2    

In [6]:
train_X, train_Y = window_torch(train, x_size, shift, x_columns, y_columns)
train_dataset = torch.utils.data.TensorDataset(train_X, train_Y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=len(train_X), shuffle=False) 

''''test_X, test_Y = window_torch(test, x_size, shift, x_columns, y_columns)
test_dataset = torch.utils.data.TensorDataset(test_X, test_Y)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False) 

val_X, val_Y = window_torch(val, x_size, shift, x_columns, y_columns)
val_dataset = torch.utils.data.TensorDataset(val_X, val_Y)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)'''

"'test_X, test_Y = window_torch(test, x_size, shift, x_columns, y_columns)\ntest_dataset = torch.utils.data.TensorDataset(test_X, test_Y)\ntest_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False) \n\nval_X, val_Y = window_torch(val, x_size, shift, x_columns, y_columns)\nval_dataset = torch.utils.data.TensorDataset(val_X, val_Y)\nval_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)"

In [7]:
print(train_X.size())
print(len(list(train_X)))
print(len(train))
print(len(train_X))
print(train_X[0])
print(train_X[12000])

torch.Size([89859, 10, 5])
89859
52
89859
tensor([[ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000],
        [ 27.6360, 986.0000,  13.0000,   2.9200,   5.0000]])
tensor([[ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000],
        [ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000],
        [ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000],
        [ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000],
        [ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000],
        [ 27.7600, 944.0000,  13.0000,   2.8200,   4.0000

In [ ]:
#train_x = train_X.reshape(5036540,5)
#train_y = train_Y.reshape(5036540,5)
#print(train_x)

In [21]:


num_x_columes = len(x_columns)

#model = nn.Linear(1,1)
model = nn.Linear(num_x_columes,1)
loss_fn = nn.MSELoss()
#loss_fn= nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
print(num_x_columes)

5


In [24]:
def Lrmodel(num_epochs, model, optimizer, train_dl, loss_fn):
    for epoch in range(num_epochs):
        '''pred = model(train_X)
        loss = loss_fn(pred, train_Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()'''
        
        for xb, yb in train_dl:
            #print("x: ", xb)
            #print("y: ", yb)
            pred = model(xb)
            loss = loss_fn(pred, yb)
            

            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
        
        print('Epochs : [{}]/[{}],  MSE Loss : {} '.format(epoch + 1, num_epochs, loss.item()))

In [25]:
learning_rate = 0.0000001

num_epochs = 500

Lrmodel(num_epochs, model, optimizer, train_loader, loss_fn)

Epochs : [1]/[500],  MSE Loss : 43209.65234375 
Epochs : [2]/[500],  MSE Loss : 40218.73828125 
Epochs : [3]/[500],  MSE Loss : 37436.3125 
Epochs : [4]/[500],  MSE Loss : 34845.9609375 
Epochs : [5]/[500],  MSE Loss : 32437.185546875 
Epochs : [6]/[500],  MSE Loss : 30192.73828125 
Epochs : [7]/[500],  MSE Loss : 28104.5390625 
Epochs : [8]/[500],  MSE Loss : 26160.931640625 
Epochs : [9]/[500],  MSE Loss : 24351.32421875 
Epochs : [10]/[500],  MSE Loss : 22666.15625 
Epochs : [11]/[500],  MSE Loss : 21097.9765625 
Epochs : [12]/[500],  MSE Loss : 19638.298828125 
Epochs : [13]/[500],  MSE Loss : 18280.857421875 
Epochs : [14]/[500],  MSE Loss : 17015.4609375 
Epochs : [15]/[500],  MSE Loss : 15837.9189453125 
Epochs : [16]/[500],  MSE Loss : 14742.5732421875 
Epochs : [17]/[500],  MSE Loss : 13723.5712890625 
Epochs : [18]/[500],  MSE Loss : 12774.048828125 
Epochs : [19]/[500],  MSE Loss : 11891.2021484375 
Epochs : [20]/[500],  MSE Loss : 11069.1689453125 
Epochs : [21]/[500],  MSE

Epochs : [159]/[500],  MSE Loss : 8.208285331726074 
Epochs : [160]/[500],  MSE Loss : 8.172418594360352 
Epochs : [161]/[500],  MSE Loss : 8.139031410217285 
Epochs : [162]/[500],  MSE Loss : 8.107949256896973 
Epochs : [163]/[500],  MSE Loss : 8.07901668548584 
Epochs : [164]/[500],  MSE Loss : 8.052084922790527 
Epochs : [165]/[500],  MSE Loss : 8.027013778686523 
Epochs : [166]/[500],  MSE Loss : 8.003674507141113 
Epochs : [167]/[500],  MSE Loss : 7.9819440841674805 
Epochs : [168]/[500],  MSE Loss : 7.96171760559082 
Epochs : [169]/[500],  MSE Loss : 7.942885398864746 
Epochs : [170]/[500],  MSE Loss : 7.925356864929199 
Epochs : [171]/[500],  MSE Loss : 7.909038543701172 
Epochs : [172]/[500],  MSE Loss : 7.8938469886779785 
Epochs : [173]/[500],  MSE Loss : 7.879702091217041 
Epochs : [174]/[500],  MSE Loss : 7.866534233093262 
Epochs : [175]/[500],  MSE Loss : 7.854276180267334 
Epochs : [176]/[500],  MSE Loss : 7.842862606048584 
Epochs : [177]/[500],  MSE Loss : 7.8322362899

Epochs : [314]/[500],  MSE Loss : 7.683971881866455 
Epochs : [315]/[500],  MSE Loss : 7.6839280128479 
Epochs : [316]/[500],  MSE Loss : 7.683884620666504 
Epochs : [317]/[500],  MSE Loss : 7.683840274810791 
Epochs : [318]/[500],  MSE Loss : 7.6837968826293945 
Epochs : [319]/[500],  MSE Loss : 7.683753490447998 
Epochs : [320]/[500],  MSE Loss : 7.683709621429443 
Epochs : [321]/[500],  MSE Loss : 7.683665752410889 
Epochs : [322]/[500],  MSE Loss : 7.683621406555176 
Epochs : [323]/[500],  MSE Loss : 7.6835784912109375 
Epochs : [324]/[500],  MSE Loss : 7.683535099029541 
Epochs : [325]/[500],  MSE Loss : 7.683490753173828 
Epochs : [326]/[500],  MSE Loss : 7.68344783782959 
Epochs : [327]/[500],  MSE Loss : 7.683404445648193 
Epochs : [328]/[500],  MSE Loss : 7.683361530303955 
Epochs : [329]/[500],  MSE Loss : 7.6833176612854 
Epochs : [330]/[500],  MSE Loss : 7.683274745941162 
Epochs : [331]/[500],  MSE Loss : 7.683231353759766 
Epochs : [332]/[500],  MSE Loss : 7.6831879615783

Epochs : [469]/[500],  MSE Loss : 7.677268981933594 
Epochs : [470]/[500],  MSE Loss : 7.6772260665893555 
Epochs : [471]/[500],  MSE Loss : 7.677183151245117 
Epochs : [472]/[500],  MSE Loss : 7.677139759063721 
Epochs : [473]/[500],  MSE Loss : 7.677096366882324 
Epochs : [474]/[500],  MSE Loss : 7.677052974700928 
Epochs : [475]/[500],  MSE Loss : 7.677010536193848 
Epochs : [476]/[500],  MSE Loss : 7.676967144012451 
Epochs : [477]/[500],  MSE Loss : 7.676923751831055 
Epochs : [478]/[500],  MSE Loss : 7.676880359649658 
Epochs : [479]/[500],  MSE Loss : 7.676837921142578 
Epochs : [480]/[500],  MSE Loss : 7.676793575286865 
Epochs : [481]/[500],  MSE Loss : 7.676750659942627 
Epochs : [482]/[500],  MSE Loss : 7.676707744598389 
Epochs : [483]/[500],  MSE Loss : 7.676664352416992 
Epochs : [484]/[500],  MSE Loss : 7.676621913909912 
Epochs : [485]/[500],  MSE Loss : 7.676578998565674 
Epochs : [486]/[500],  MSE Loss : 7.676535606384277 
Epochs : [487]/[500],  MSE Loss : 7.676492214

In [ ]:

print(list(model.parameters()))
# https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_linear_regression/
# cuda 사용 링크

# csv 51개
# Epochs = 30 , a = 0.0000001, 타임윈도우 x_size = 2, shift = 1 일때 MSELoss : 6.9482
# Epochs = 30 , a = 0.0000001, 타임윈도우 x_size = 10, shift = 5 일때 MSELoss : 9.5871
# Epochs = 200, a = 0.00000001, 타임윈도우(10,5), MSELoss : 21.0888
# Epochs = 500, a = 0.0000001, 타임윈도우(10,5), MSELoss : 7.5659


### 